In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-06-30 14:31:51,028 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [38]:
kwargs = {
    "game": "iRacing",
    # "track": "oschersleben gp",
    # "track": "aragon gp",
    "track": "jerez gp",
    "car": "Ferrari 488 GT3 Evo 2020",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
# laps = list(laps)
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-06-30 16:50:14,375 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-06-30 16:50:14,375 INFO   track.id 61 car.id 9
2023-06-30 16:50:14,375 INFO   session 1688133673 lap.id 171188 number 3
2023-06-30 16:50:14,376 INFO   length 4369 time 103.2424 valid True
2023-06-30 16:50:14,376 INFO   start 2023-06-30 16:04:45.405959+00:00 end 2023-06-30 16:06:28.669162+00:00
2023-06-30 16:50:14,397 ERROR No data found for 1688133673 lap 3
2023-06-30 16:50:14,397 INFO No data found for lap in fast_laps bucket, trying in default bucket
2023-06-30 16:50:14,398 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-06-30 16:50:14,398 INFO   track.id 61 car.id 9
2023-06-30 16:50:14,398 INFO   session 1688133673 lap.id 171188 number 3
2023-06-30 16:50:14,399 INFO   length 4369 time 103.2424 valid True
2023-06-30 16:50:14,399 INFO   start 2023-06-30 16:04:45.405959+00:00 end 2023-06-30 16:06:28.669162+00:00
2023-06-30 16:50:14,912 INFO Fetching tele

In [42]:
# for i, df in enumerate(data_frames):
#     fig = lap_fig(df, full_range=True, columns=["Throttle"])
#     fig.show()
sector_start_end, df_max = fast_lap_analyzer.extract_sectors(data_frames)
display(sector_start_end)
fig = lap_fig(df_max, full_range=True, columns=["Throttle"])
# add a vertical line for each sector
for i, sector in enumerate(sector_start_end):
    fig_add_shape(fig, x0=sector["start"], x1=sector["start"], color="blue")
    fig_add_shape(fig, x0=sector["end"], x1=sector["end"], color="red")
fig.show()

[{'start': 4242, 'end': 350, 'length': 477},
 {'start': 351, 'end': 673, 'length': 322},
 {'start': 674, 'end': 985, 'length': 311},
 {'start': 986, 'end': 1604, 'length': 618},
 {'start': 1605, 'end': 2277, 'length': 672},
 {'start': 2278, 'end': 2747, 'length': 469},
 {'start': 2748, 'end': 3131, 'length': 383},
 {'start': 3132, 'end': 3605, 'length': 473},
 {'start': 3606, 'end': 4241, 'length': 635}]

In [45]:
segments, used_laps = fast_lap_analyzer.extract_segments(sector_start_end, data_frames, laps_with_telemetry, df_max)
for i in range(len(segments)):
    segment = segments[i]
    fig = lap_fig(segment.telemetry_for_fig(), columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        fig_add_features(fig, segment.brake_features())
    throttle_features = segment.throttle_features()
    if throttle_features:
        fig_add_features(fig, segment.throttle_features(), color="green")
    fig.show()